# NSLKDD Classification using Autoencoders

## Imports

In [15]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, normalize
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

# !pip install tensorflow==2.1.0
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Conv1D, MaxPool1D
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.losses import KLDivergence, mean_squared_error

import keras
from keras.utils import np_utils

## Data Preprocessing

In [16]:
df = pd.read_csv('../input/nslkdd/kdd_train.csv')
df.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [17]:
le = LabelEncoder()

df['labels'] = le.fit_transform(df['labels'])
df['protocol_type'] = le.fit_transform(df['protocol_type'])
df['service'] = le.fit_transform(df['service'])
df['flag'] = le.fit_transform(df['flag'])

In [18]:
x = df.drop('labels', axis=1)
y = df.loc[:, ['labels']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_test = sc.fit_transform(X_test)
X_test = pd.DataFrame(X_test)

In [19]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

(113375, 23)

### Feature Reduction

In [21]:
DTC = DecisionTreeClassifier()
feature_dim = 30
rfe = RFE(DTC, n_features_to_select= feature_dim).fit(X_train,y_train)
indices = np.where(rfe.support_==True)[0]
features = X_train.columns.values[indices]
X_train, X_test = X_train[features], X_test[features]

## Utils

### Custom Loss

In [ ]:
class HybridHuberLoss(tf.keras.losses.Loss):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        kld = KLDivergence()
        kld_loss = kld(y_true, y_pred)
        return tf.where(mse<kld_loss, kld_loss, mse)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Learning Rate Schedulers

In [ ]:
def LRschedulerCLF(epoch, lr=0.01):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def LRschedulerAE(epoch):
    initial_lrate = 0.01
    drop = 0.005
    epochs_drop = 5.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
    return lrate
    
clf_lr = tf.keras.callbacks.LearningRateScheduler(LRschedulerCLF)
ae_lr = tf.keras.callbacks.LearningRateScheduler(LRschedulerAE)

# Autoencoder

In [22]:
input_dim = X_train.shape[1]
encoding_dim = 16
ae_input_layer = Input(shape=(input_dim, ))

enc = Dense(32, activation="swish", activity_regularizer=regularizers.l1(10e-5))(ae_input_layer)
enc = BatchNormalization()(enc)
enc = Dense(encoding_dim, activation="swish")(enc)

dec = BatchNormalization()(enc)
dec = Dense(32, activation="swish")(dec)
dec = BatchNormalization()(enc)
dec = Dense(input_dim, activation="swish")(dec)

autoencoder = Model(inputs=ae_input_layer, outputs=dec)
encoder = Model(inputs=ae_input_layer, outputs=enc)

In [23]:
ae_epoch = 15
batch_size = 32
autoencoder.compile(optimizer='adam', loss="mean_squared_error", metrics=['accuracy'])

In [24]:
history = autoencoder.fit(X_train, X_train, 
                    epochs=ae_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    callbacks = [ae_lr],
                    verbose=1).history

Epoch 1/15
3543/3543 [==============================] - 8s 2ms/step - loss: 0.6441 - accuracy: 0.3969 - val_loss: 0.9824 - val_accuracy: 0.6419
Epoch 2/15
3543/3543 [==============================] - 7s 2ms/step - loss: 0.3830 - accuracy: 0.5468 - val_loss: 1.5027 - val_accuracy: 0.5792
Epoch 3/15
3543/3543 [==============================] - 7s 2ms/step - loss: 0.3567 - accuracy: 0.5490 - val_loss: 1.7271 - val_accuracy: 0.6802
Epoch 4/15
3543/3543 [==============================] - 7s 2ms/step - loss: 0.3364 - accuracy: 0.5486 - val_loss: 2.6176 - val_accuracy: 0.5432
Epoch 5/15
3543/3543 [==============================] - 7s 2ms/step - loss: 0.4441 - accuracy: 0.5535 - val_loss: 1.9979 - val_accuracy: 0.6394
Epoch 6/15
3543/3543 [==============================] - 7s 2ms/step - loss: 0.3855 - accuracy: 0.5485 - val_loss: 1.6745 - val_accuracy: 0.8647
Epoch 7/15
3543/3543 [==============================] - 7s 2ms/step - loss: 0.3828 - accuracy: 0.5547 - val_loss: 1.8285 - val_accuracy:

# Classifier + Autoencoder

In [25]:
encoder.trainable = False

input_layer = Input(shape=(feature_dim, ))

encoding = encoder(input_layer, training=False)

layer1 = Dense(64, activation="relu")(encoding)
layer1 = BatchNormalization()(layer1)
layer1 = Dropout(0.2)(layer1)

layer2 = Dense(64, activation="relu")(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Dropout(0.3)(layer2)

layer3 = Dense(128, activation="relu")(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Dropout(0.3)(layer3)

layer4 = Dense(128, activation="relu")(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Dropout(0.2)(layer4)

output_layer = Dense(23, activation="softmax")(layer4)

classifier = Model(inputs=input_layer ,outputs=output_layer)

In [26]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'AUC'])

In [27]:
history = classifier.fit(X_train, y_train,
                    epochs=15,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, y_test),
                    callbacks=[ae_lr],
                    verbose=1).history

Epoch 1/10
3543/3543 [==============================] - 12s 3ms/step - loss: 0.3748 - accuracy: 0.9073
Epoch 2/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.1028 - accuracy: 0.9705
Epoch 3/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0832 - accuracy: 0.9762
Epoch 4/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0751 - accuracy: 0.9788
Epoch 5/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0711 - accuracy: 0.9799
Epoch 6/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0652 - accuracy: 0.9815
Epoch 7/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0631 - accuracy: 0.9819
Epoch 8/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0590 - accuracy: 0.9833
Epoch 9/10
3543/3543 [==============================] - 11s 3ms/step - loss: 0.0574 - accuracy: 0.9837
Epoch 10/10
3543/3543 [==============================] - 11s 3ms/step - l

## Model Evaluation

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

In [192]:
y_preds = classifier.predict(X_test)
y_preds = np.round_(y_preds)

In [193]:
print(precision_score(y_test, y_preds, average='micro', zero_division=0))
print(precision_score(y_test, y_preds, average='weighted', zero_division=0))


In [ ]:
print(f1_score(y_test, y_preds, average='micro', zero_division=0))
print(f1_score(y_test, y_preds, average='weighted', zero_division=0))
